In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

In [2]:
# read csv data as dataframe
data = pd.read_csv('/home/iik/Desktop/tiria/data/2021-07-20_21-57_results.csv', delimiter=';')
data = data.rename(columns={'Schueler-ID': 'child_id'})
data.head()

,Klasse,Stufe,Lehrer,ID,child_id,Test,Aufgabe,Gewaehlt1,Gewaehlt2,Richtig,Ergebnis,Start,Ende,Dauer,GesamtRichtig,GesamtFalsch,Level,GesamtLevel
0,1a,1,Müller,101,1,1,Trial 1,Büffel,Delfin,"Dachs,Delfin",False,07.06.2021 17:36:30,07.06.2021 17:37:24,00:00:54,8,2,C,C
1,1a,1,Müller,101,1,1,Trial 2,Wal,Tiger,"Wildschwein,Wal",False,07.06.2021 17:36:30,07.06.2021 17:37:24,00:00:54,8,2,C,C
2,1a,1,Müller,101,1,1,1,Ameise,Fuchs,"Ameise,Adler",False,07.06.2021 17:36:30,07.06.2021 17:37:24,00:00:54,8,2,C,C
3,1a,1,Müller,101,1,1,2,Baum,Biene,"Baum,Biene",True,07.06.2021 17:36:30,07.06.2021 17:37:24,00:00:54,8,2,C,C
4,1a,1,Müller,101,1,1,3,Erdbeere,Ente,"Erdbeere,Ente",True,07.06.2021 17:36:30,07.06.2021 17:37:24,00:00:54,8,2,C,C


In [3]:
# rename variables
data = data.rename(columns={'Schueler-ID': 'child_id', 
                            'Gewaehlt1': 'choice_1',
                            'Gewaehlt2':'choice_2',
                            'GesamtRichtig':'Gesamt_Richtig',
                            'GesamtFalsch':'Gesamt_Falsch',
                            'GesamtLevel': 'Gesamt_Level'})

# delete the tutorial tasks from df (trial 1 & trial 2) as well as test trials (which is ID == 101 and ==202) as they should not be included in the analysis
data = data.loc[~((data['Aufgabe']=='Trial 1') | (data['Aufgabe']=='Trial 2'))]

# delete all test trials 
data = data.loc[~((data.ID ==101) | (data.ID ==202))]

# transform datatype from variable 'Aufgabe' to type integer
data.Aufgabe = data.Aufgabe.astype(int)

# Create Date time object
data.Start = pd.to_datetime(data.Start, format='%d.%m.%Y %H:%M:%S')
data.Ende = pd.to_datetime(data.Ende, format='%d.%m.%Y %H:%M:%S')

data.insert(0, 'unique_id', data.ID.astype(str)+'_'+ data.child_id.astype(str))

data['Ergebnis'] = data['Ergebnis'].replace({'True':1, 'False':0, 'Keine Auswahl': 99})
data['Ergebnis'] = data['Ergebnis'].astype(int)

data["item_choices"] = data["Aufgabe"].astype(str) +'_'+ data["Richtig"]
data.head()

#data.head()



,unique_id,Klasse,Stufe,Lehrer,ID,child_id,Test,Aufgabe,choice_1,choice_2,Richtig,Ergebnis,Start,Ende,Dauer,Gesamt_Richtig,Gesamt_Falsch,Level,Gesamt_Level,item_choices
1832,303_0,Fuchsbande,2,Cordula Wuttke,303,0,1,1,Ameise,Adler,"Ameise,Adler",1,2021-06-29 11:10:05,2021-06-29 11:11:08,00:01:02,10,0,C,B,"1_Ameise,Adler"
1833,303_0,Fuchsbande,2,Cordula Wuttke,303,0,1,2,Baum,Biene,"Baum,Biene",1,2021-06-29 11:10:05,2021-06-29 11:11:08,00:01:02,10,0,C,B,"2_Baum,Biene"
1834,303_0,Fuchsbande,2,Cordula Wuttke,303,0,1,3,Erdbeere,Ente,"Erdbeere,Ente",1,2021-06-29 11:10:05,2021-06-29 11:11:08,00:01:02,10,0,C,B,"3_Erdbeere,Ente"
1835,303_0,Fuchsbande,2,Cordula Wuttke,303,0,1,4,Fisch,Fliege,"Fisch,Fliege",1,2021-06-29 11:10:05,2021-06-29 11:11:08,00:01:02,10,0,C,B,"4_Fisch,Fliege"
1836,303_0,Fuchsbande,2,Cordula Wuttke,303,0,1,5,Gras,Giraffe,"Gras,Giraffe",1,2021-06-29 11:10:05,2021-06-29 11:11:08,00:01:02,10,0,C,B,"5_Gras,Giraffe"


In [4]:
set4 = data.loc[data['Test'] == 4]

test4_statements = pd.read_csv('/home/iik/Desktop/tiria/data/Waldwörter - Einstufungstest 4.csv')
test4_statements.drop(columns=['Falsch', 'Richtig '], inplace=True)
test4_statements.rename(columns={'Position im Test':'Aufgabe', 'Satz ': 'Test4_statement'}, inplace=True)
#test4_statements
result = pd.merge(set4, test4_statements, on=["Aufgabe"])
result.head()


,unique_id,Klasse,Stufe,Lehrer,ID,child_id,Test,Aufgabe,choice_1,choice_2,Richtig,Ergebnis,Start,Ende,Dauer,Gesamt_Richtig,Gesamt_Falsch,Level,Gesamt_Level,item_choices,Test4_statement,Schwierigkeit
0,303_0,Fuchsbande,2,Cordula Wuttke,303,0,4,1,True,NaN,False,0,2021-06-29 11:19:22,2021-06-29 11:20:12,00:00:50,16,14,A,B,1_False,Igel schwimmen im Bach.,1
1,303_1,Fuchsbande,2,Cordula Wuttke,303,1,4,1,False,NaN,False,1,2021-06-30 06:26:14,2021-06-30 06:28:47,00:02:32,12,5,B,C,1_False,Igel schwimmen im Bach.,1
2,303_2,Fuchsbande,2,Cordula Wuttke,303,2,4,1,False,NaN,False,1,2021-06-30 06:22:08,2021-06-30 06:23:43,00:01:35,28,2,C,C,1_False,Igel schwimmen im Bach.,1
3,303_3,Fuchsbande,2,Cordula Wuttke,303,3,4,1,True,NaN,False,0,2021-06-30 06:28:29,2021-06-30 06:30:52,00:02:22,28,2,C,C,1_False,Igel schwimmen im Bach.,1
4,303_4,Fuchsbande,2,Cordula Wuttke,303,4,4,1,False,NaN,False,1,2021-06-30 06:26:54,2021-06-30 06:29:28,00:02:33,17,9,B,B,1_False,Igel schwimmen im Bach.,1


In [5]:
data['idx'] = data.groupby('unique_id').cumcount()
data = data.pivot_table(index=['Test', 'Stufe', "unique_id", 'Level', 'Gesamt_Level'], 
                    columns=['item_choices'], 
                    values='Ergebnis').reset_index()

data = pd.DataFrame(data.to_records())
data = data.drop(columns=['index'])
data.head()

,Test,Stufe,unique_id,Level,Gesamt_Level,10_Apfel,"10_Eichhörnchen,Delfin","10_Pinguin,Papagei",10_True,11_False,11_Gans,12_Katze,12_True,13_Pilz,13_True,14_Ameise,14_False,15_False,15_Löwe,16_Schaf,16_True,17_False,17_Papagei,18_Ananas,18_True,19_Eichhörnchen,19_True,1_Adler,"1_Ameise,Adler","1_Büffel,Vogel",1_False,20_Büffel,20_False,21_Bär,21_True,22_Fliege,22_True,23_Lama,23_True,24_Eisbär,24_False,25_Biene,25_True,26_Ast,26_False,27_Hirsch,27_True,28_False,28_Schmetterling,29_Blatt,29_False,"2_Baum,Biene",2_Hase,"2_Lama,Zebra",2_True,30_Kirsche,30_True,31_Liege,32_Maus,33_Pflanze,34_Delfin,35_Berg,36_Honig,37_Schwein,38_Wildschwein,39_Regenwurm,3_Affe,"3_Erdbeere,Ente",3_True,"3_Wildschwein,Pinguin",40_Möwe,41_Huhn,42_Maulwurf,43_Zebra,44_Reh,45_Biber,46_Birne,47_Nuss,48_Mund,49_Käfer,4_Baum,4_False,"4_Fisch,Fliege","4_Schaf,Maulwurf",50_Spinne,5_Esel,"5_Gras,Giraffe","5_Käfer,Eisbär",5_True,"6_Ananas,Gans",6_Ente,6_False,"6_Hase,Hirsch","7_Biene,Affe",7_False,"7_Kirsche,Käfer",7_Wal,"8_Maus,Maulwurf","8_Regenwurm,Stamm",8_True,8_Wolf,"9_Elefant,Blatt",9_False,"9_Regenwurm,Rind",9_Sonne
0,1,1,404_0,C,C,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,1,1,404_1,A,A,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,1,1,404_10,B,B,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,1,1,404_11,A,A,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,1,1,404_12,B,C,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN


### Create Dataset for subtest 4

In [6]:
result["item_choices"] = result["Aufgabe"].astype(str) +'_'+ result["Test4_statement"]
result.head()
result['idx'] = result.groupby('unique_id').cumcount()
result = result.pivot_table(index=['Test', 'Stufe', "unique_id", 'Level', 'Gesamt_Level'],#, 'Schwierigkeit'], 
                    columns=['item_choices'], 
                    values='Ergebnis').reset_index()

result = pd.DataFrame(result.to_records())
result = result.drop(columns=['index'])
table4_out = result.to_csv('/home/iik/Desktop/tiria/data/test4_statements.csv', index=False)
print('ok')

ok


In [7]:
# drop dublicates


In [8]:
table_out = data.to_csv('/home/iik/Desktop/tiria/data/allTests.csv', index=False)
print('ok')

ok


In [9]:
dict_test_df={}
test_name = data.Test.unique()
for test in test_name:
    df_test = data.loc[data['Test'] == test]
    dict_test_df[test]= df_test

In [10]:
listTest=[1,2,3,4]
for t in listTest:
    test = dict_test_df.get(t)
    filepath = '/home/iik/Desktop/tiria/data/test_'+ str(t)+'.csv'
    test_out = test.to_csv(filepath)